# **04. 결제 예측 모델링**
--------

## **목적**
* ### 결제 행동을 예측하는 모델을 구축하고 평가한다.
* ### 최적의 파라메터를 찾는 방법을 실습한다.
<br>

## **목차**
* ###  문제 정의 및 가설
* ###  분석 Frame 구성
* ###  데이터 전처리
* ###  데이터 분석
* ###  리포트 작성

# **04-1. 문제 정의 및 가설**
--------

### 여러 번 강조해도 지나침이 없을 정도로, 문제를 명확히 정의/설정하고 목표를 세우는 것은 매우 중요하다. 만약 목표가 틀리다면, 아무리 열심히 수고하더라도 큰 의미가 없기 때문이다.(물론 회사 입장에서 그렇다. 개인 입장에서는 경험치 획득에 도움이 될 수는 있다) 이번에는 마케팅팀에서 업무 요청이 왔다고 가정한다. 마케팅팀이 관심을 두는 것은 회원의 행동 패턴을 근거로, 결제로 전환할 유저를 모델을 통해 예측하는 것이 가능한지 여부이다.

## **배경**
### 배경 및 상황에 대해 조금더 자세히 살펴보자. 마케팅팀은 주로 예산을 집행하고 이에 따른 결과(성과)를 통해 예산을 효율화하는 것이 관심을 두고 있다. 이를 위해 흔히 알려진 전략이 STP이다. 즉 유저/소비자를 세분화해 그룹으로 나누고, 각 그룹에 맞는 개인화 메시지를 전달하거나 그룹의 우선순위에 따라 예산 및 접근 방식에 차등을 두는 방식을 주로 이용한다. 이번 사례에서는 결제자의 수를 증가시키기 위한 비즈니스 전략을 달성하기 위해 마케팅팀으로부터 결제자 예측 모델링 구축 관련한 업무 요청이 왔다고 가정하고 진행한다.

* ### 결제자수의 증가에 대한 비즈니스 요구 발생 (마케팅팀 KPI)
* ### 유저의 행동을 기반으로, 결제 가능성이 높은 유저를 선별하여 해당 유저를 위주로 타깃팅 (광고, 프로모션 등)
* ### 데이터 분석가 뿐만 아니라, 여러 팀의 협업이 필요한 상황

## **목적**
### 이번 사례의 경우 아래와 같이 데이터 분석가의 목적을 설정할 수 있다. 단, 지금 사례와 같이 여러 팀이 협업해서 프로젝트를 진행하는 경우라면 전체 프로젝트의 목적과 각 팀 혹은 파트 별로의 목적을 충분한 논의를 통해 설정하는 것이 필요하다.

* ### 결제자로 전환할 가능성이 높은 유저를 판별하는 모델 구축
* ### 예측 모델의 성능을 측정할 수 있는 지표를 정의

![screenshot](.\img\28.png "28_Sh")

## **가설**
### 이번 사례의 경우 가설을 설정하기 위해 도메인 지식이 많이 요구된다. 도메인 지식과 전문가의 조언 등을 통해 결제와 관련 있는 행동 변수를 사전에 이해하고 리스트업하는 것이 유용하다. 만약 이러한 도움을 받기 어려운 상황이라면, 분석가의 상상력과 직감을 이용해 주요 변수들을 생각해놓고 탐색적 데이터 분석을 통해 Feature Selection을 하는 것이 필요하다.

* ### 가설 1: Open, Edit/Save, Export와 같은 문서 이용행동 관련 변수가 모델 구축에 유의미한 변수일 것
* ### 가설 2: 문서 사용 트래픽 및 일주일 간 방문일수 역시 결제 행동와 관련 있는 예측 변수일 것

## **Expected Output**
* ### 타깃팅 유저 정보 테이블 (RDB, 일별 배치)
    * ### Schema: 날짜, 타깃팅 대상 유저의 아이디, 결제 확률, 결제 예상 여부 (결제 확률 50% 이상일 경우 True, 이하일 경우 False)
* ### 타깃팅 결과 테이블 (RDB, 일별 배치)
    * ### Schema: 날짜, 타깃 유저 아이디, 타깃 액티비티, 결제 여부(기존 3일내)
* ### 결제 정보 테이블 (기존 Hadoop 저장 테이블)
    * ### Schema: 날짜, 전체 결제 유저 아이디, 결제일
* ### 모델 구축 과정 문서(수식) 및 코드
* ### 모델 성과 측정 대시보드/리포트 (Recall, Precision, AUC, F1-score)


# **04-2. 문제 정의 및 가설**
--------

### 우선 일반적인 모델링 구축을 위한 프로세스를 먼저 살펴보자.

![screenshot](.\img\29.png "29_Sh")

### 이번 프로젝트 역시 일반적인 프로세스와 거의 유사하게 진행될 것이다. 리포트 및 공유 문서에 위와 같은 프레임 이미지를 협업자에게 보여준다면 모델링 프로세스에 대한 이해가 높아질 것으로 기대할 수 있다. 위 이미지를 참고하여 이번 프로젝트의 프레임을 구성해보자.

* ### **데이터 수집**
    * ### 결제 바로 전, 유저의 행동 패턴을 기술할 수 있는 로그 항목을 수집 (문서 오픈, 편집 등)
* ### **데이터 추출**
    * ### Extraction, Preprocessing (SQL 필터, 조인 등)
* ### **데이터 전처리**
    * ### Feature Engineering (분포 변환, PCA, 결측치 및 이상치 처리 등)
* ### **모델 구축 및 파라메터 설정**
    * ### Classification Models (Logistic Regression, Random Forest, etc)
    * ### Cross Validation, Grid Search, Pipeline
* ### **모델 평가**
    * ### Precision, Recall, F1-score

## **Logic Tree**

### 지난 시간에 배운 로직 트리를 이용해 전체적인 흐름을 구성하도록 하자. 모델을 구축하고 평가하는 분석가의 role은 일반적으로 큰 프로젝트 안에서 하나의 부분(part)를 맡게 된다. 비즈니스 환경에서는 단순히 모델을 구축하는 것이 목적이 되기 보다, 별도의 목표를 달성하기 위한 하나의 수단에 지나지 않는 경우가 많다.

### 따라서 전체 프로젝트(숲)을 보고 상황에 맞게 도구(나무)를 활용하려는 노력이 필요하다. 로직트리를 구성할 때는 전체적인 프로젝트를 고려하면서 짜는 것이 유용할 수 있다.

![screenshot](.\img\30.png "30_Sh")

[실습파일](https://github.com/songhunhwa/songhunhwa.github.com/tree/master/tutorial/tutorial_03 "로직 트리 작성")

## **추가 고려 사항**
### 대부분의 분석 프로젝트도 마찬가지지만, 예측 모델링을 활용하는 프로젝트의 경우 특히 타팀과 팀원간 많은 협업이 요구된다. 분석가가 관여하는 부분은 데이터를 입수해 모델을 구축하고 평가 과정을 통해 최종적인 모델을 만드는 과정이며, 분석가가 만든 프로토타입 코드를 엔지니이와 개발자가 제품에 녹이는 과정도 요구된다. 또 기획 및 운영 등 여러 팀이 관여하게 되므로 전체적인 PM의 역할이 특히 더 중요하다고 할 수 있다.

* ### 전체적인 그리고 세부적인 일정이 어떻게 되는가? 모델링과 관련된 일정을 맞출 수 있는가?
* ### 각 팀 및 담당자의 역할은 정확히 무엇인가?
* ### 엔지니어 및 개발 담당은 누구이며, 어떠한 사항이 논의되어야 하는가? 분석 및 개발 언어는? 제품 내 모델 및 데이터 처리 방식은?
* ### 정확한 목적은 무엇이며 목표 달성 수준은 어떻게 정의할 것인가?
* ### 필요한 데이터가 무엇인가? 확보가 가능한가?

# **04-3. 데이터 전처리**
------

### 모든 데이터 분석 프로젝트에서 데이터 전처리는 반드시 거쳐야 하는 과정이다. 대부분의 데이터 분석가가 좋아하지 않는 과정이지만, 분석 결과/인사이트와 모델 성능에 직접적인 영향을 미치는 과정이기 때문에 중요하게 다루어지는 과정이다. 한 설문조사에 의하면, 분석가의 80% 시간을 데이터 수집 및 전처리에 사용한다고 하니, 얼마나 중요한 과정인지 짐작할 수 있다. 물론 지루하고 반복 작업의 연속이기 때문에 시간이 많이 들어가는 측면도 있을 것이다.

![screenshot](.\img\31.png "31_Sh")

### 지난 시간에 간단히 언급한 대로, 실무에 사용되는 데이터셋은 바로 분석이 불가능할 정도로 지저분(messy)하다. 분석이 가능한 상태로 만들기 위해 아래와 같은 전처리 방식이 자주 사용된다. 모든 강의에 걸쳐서 전처리 단계는 중요하게 그리고 반복적으로 다루어질 예정이다.

## **데이터 불러오기 및 인덱스 지정**
### 데이터를 읽어 오고 가장 먼저 할일은 첫 행 몇개와 마지막 행을 살펴보는 것이다. 그리고 유니크 식별값을 인덱스로 지정하고 dataframe 각 컬럼의 타입과 결측치 등을 파악하는 것이다.
```
# 데이터프레임 읽고 초반, 후반 행 확인하기
df = pd.read_csv("testset.csv", index_col=0)
df.head()
df.tail()

# 인덱스 지정
df.set_index("iduser", inplace=True)

# 컬럼별 type 확인 및 결측치 확인
df.info()
df.isnull().sum()
```

## **결측치 처리**

### 결측치 처리는 1) 결측치 사례 제거 2) 수치형의 경우 평균이나 중앙치로 대체(imputation)하거나 범주형인 경우 mode 값으로 대체 3) 간단한 예측 모델로 대체하는 방식이 일반적으로 이용된다. 가장 쉬운 방법은 Null이 포함 행 혹은 일부 행을 제거하는 것이다. 수집된 사례(observation)이 많다면 이 방법을 사용하는 것이 가능하다. 만약 샘플수가 충분하지 않을 경우, Pandas의 fillna() 명령어로 Null 값을 채우는 것이 가능하다. 연속형인 경우 Mean이나 Median을 이용하고 명목형인 경우 Mode(최빈치)나 예측 모형을 통해 Null 값을 대체할 수 있다.

### 데이터셋을 읽었다면, Missing Value 파악을 위해 df.info() 가장 처음에 이용하는 것을 추천한다. 만약 np.nan으로 적절히 missing value로 불러왔다면 info() 이용 가능하다. 만약 '', ' ' 이런식의 공백이나 다른 방식으로 처리되어 있다면, 모두 repalce 처리해줘야 한다. info()를 실행했을 때, 누가봐도 float or int 인데 object(string)으로 되어 있다면 이런 사레가 포함될 가능성이 높다.

## **결측치를 처리할 때 고려할 점**

### 결측치를 처리할 경우에도 도메인 지식은 유용하게 사용된다. 인적, 기계적 원인임이 판명되면, 협업자와 지속적으로 노력해 결측치를 사전에 발생하지 않도록 조치하는 것이 좋다. 수치형인 경우 의미상으로 0으로 메꾸는 것이 맞는지 아니면 평균이나 중앙치가 맞는지 등은 데이터에 대한 배경지식이 있는 경우, 보다 적절한 의사결정을 할 수 있다.

### 예를 들어 viewCount가 1이상인데, edit, export가 missing인 경우 (도메인 지식을 통해) 0으로 메꾸는 것이 가능하다. View 가 다른 행동에 선행하는 개념이기 때문에 위와 같은 의사결정이 가능하다.

* ### NA 와 Null 차이점 (R에서만 구분되는 개념, 파이썬에서는 numpy의 NaN만 이용, 가끔 pure python에서 None을 볼 수 있음, None = empty)
    * ###  NA: Not Available (does not exist, missing)
    * ###  Null: empty(null) object
    * ### NaN: Not a Number (python)
    * ### reference: https://www.r-bloggers.com/r-na-vs-null/

### 특히 숫자 0과 null 과 같은 결측치는 완전히 다른 개념이니 유의해야 한다. 만약 target(group)에 결측치가 있다면 imputation이 아닌 drop 으로 처리하도록 한다.
* ### 0: -1과 1 사이의 가운데 숫자(정수)
* ### null: 미지의 값

```
# 결측치 부분을 메꾸는 방법
test['viewCount'] = test['viewCount'].fillna(test.viewCount.mean())

# 만약 결측치가 문자열 스페이스(' ')로 되어 있다면, np.nan으로 바꾸어 Pandas 라이브러리가 인식할수 있도록 변환
test.viewCount = test.viewCount.replace('', np.nan)

# 결측치를 제거하는 방법
test.dropna(how='all').head() # 한 행이 모두 missing value이면 제거
test.dropna(how='any').head() # 한 행에서 하나라도 missing value가 있으면 제거
```

## **이상치 처리**
### 일반적으로 1) 표준점수로 변환 후 -3 이하 및 +3 제거 2) IQR 방식 3) 도메인 지식 이용하거나 Binning 처리하는 방식이 이용된다. 표준점수 이용할 경우 평균이 0, 표준편차가 1인 분포로 변환한후 +3 이상이거나 -3 이하인 경우 극단치로 처리한다.


![screenshot](.\img\32.png "32_Sh")

```
# 표준점수 기반 예제 코드
def std_based_outlier(df):
    for i in range(0, len(df.iloc[1])): 
        df.iloc[:,i] = df.iloc[:,i].replace(0, np.NaN) # optional
        df = df[~(np.abs(df.iloc[:,i] - df.iloc[:,i].mean()) > (3*df.iloc[:,i].std()))].fillna(0)
```

### IQR 방식은 75% percentile + 1.5 * IQR 이상이거나 25 percentile - 1.5 * IQR 이하인 경우 극단치로 처리하는 방식이다. 이해하기 쉽고 적용하기 쉬운 편이지만, 경우에 따라 너무 많은 사례들이 극단치로 고려되는 경우가 있다.

![screenshot](.\img\33.png "33_Sh")

```
# IQR 기반 예제 코드
def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return np.where((ys > upper_bound) | (ys < lower_bound))
```

## **데이터 분포 변환**
### 대부분의 모델은 변수가 특정 분포를 따른다는 가정을 기반으로 한다. 예를 들어 선형 모델의 경우, 설명 및 종속변수 모두가 정규분포와 유사할 경우 성능이 높아지는 것으로 알려져 있다. 자주 쓰이는 방법은 Log, Exp, Sqrt 등 함수를 이용해 데이터 분포를 변환하는 것이다. 

```
import math
from sklearn import preprocessing

# 특정 변수에만 함수 적용
df['X_log'] = preprocessing.scale(np.log(df['X']+1)) # 로그
df['X_sqrt'] = preprocessing.scale(np.sqrt(df['X']+1)) # 제곱근

# 데이터 프레임 전체에 함수 적용 (단, 숫자형 변수만 있어야 함)
df_log = df.apply(lambda x: np.log(x+1))  
```

### 위 방법 외에도 분포의 특성에 따라 제곱, 자연로그, 지수 등 다양한 함수가 사용될 수 있다. 가이드는 아래와 같다.

* ### left_distribution: X^3
* ### mild_left: X^2
* ### mild_right: sqrt(X)
* ### right: ln(X)
* ### servere right: 1/X

## **데이터 단위 변환**
### 데이터의 스케일(측정단위)이 다를 경우 특히 거리를 기반으로 분류하는 모델(KNN 등)에 부정적인 영향을 미치므로, 스케일링을 통해 단위를 일정하게 맞추는 작업을 진행해야 한다. 아래 방식이 주로 스케일링을 위해 쓰이는 방법이다. 대부분의 통계 분석 방법이 정규성 가정을 기반으로 하므로 완벽하지 않더라도 최대한 정규분포로 변환하는 노력이 필요하다.

* ### Scaling: 평균이 0, 분산이 1인 분포로 변환
* ### MinMax Scaling: 특정 범위 (예, 0~1)로 모든 데이터를 변환
* ### Box-Cox: 여러 k 값중 가장 작은 SSE 선택
* ### Robust_scale: median, interquartile range 사용(outlier 영향 최소화)

```
from scipy.stats import boxcox

# 변수별 scaling 적용
df['X_scale'] = preprocessing.scale(df['X']) 
df['X_minmax_scale'] = preprocessing.MinMaxScaler(df['X']
df['X_boxcox'] = preprocessing.scale(boxcox(df['X']+1)[0])
df['X_robust_scale'] = preprocessing.robust_scale(df['X'])

# 데이터 프레임 전체에 scaling 적용
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# StandardScaler
for c in df:
    df_sc[c] = StandardScaler().fit_transform(df[c].reshape(-1,1)).round(4)

# MinMaxScaler
for c in df:
    df_minmax[c] = MinMaxScaler().fit_transform(df[c].reshape(-1,1).round(4))
```

[실습파일](https://github.com/songhunhwa/songhunhwa.github.com/tree/master/tutorial/tutorial_03 "데이터 전처리")

# **04-4. 모델 생성**
------

## **모델 구축 프로세스**
### 데이터 전처리가 끝났다면, 아래와 같은 프로세스로 모델을 구축한다. 모델링 단계에 들어왔어도 데이터 전처리는 끝난 것은 아니다. 모델의 성능은 알고리즘의 차이 보다 전처리를 어떻게 했는지에 따라 더 많이 영향을 받는 것으로 알려져 있다. 따라서 모델의 성능을 측정해보고 결과가 만족스럽지 않다면, (여러 알고리즘을 사용해보는 것과 더불어) 전처리 단계를 다시 진행해야 할 수도 있다. 또 전처리를 잘하기 위해선 EDA/시각화 과정을 반복해서 진행할 필요가 있다.

![screenshot](.\img\34.png "34_Sh")

## **모델 구축전 확인사항**
### 실제 모델 구축에 들어가기 전에 아래와 같은 사항에 대해 확인하고 진행하는 것을 권장한다.

* ### 범주형, 연속형 데이터 형식의 적절성
* ### 이상치 및 결측치 처리 여부
* ### 스케일링 및 분포 변환
* ### 다중공선성 문제
* ### 예측변수(y)의 분포 imbalance 문제
* ### 변수 축소 및 파생 변수 생성

## **분류 모델**
* ### Logistic Regression
* ### Naive Bayes
* ### k-Nearest Neighbor
* ### Tree-based Model
    * ### Decision Tree
    * ### Random Forest
    * ### Gradient Boost Regression Tree
* ### SVM (Support Vector Machines)
* ### ANN (Artificial Neural Network
    * ### CNN
    * ### RNN
## **회귀 모델**
* ### Linear Regression
    * ### LASSO
    * ### Ridge


## **고려사항**
### 모델 구축을 계획하느 단계부터 아래와 같은 사항을 미리 고려해놓을 필요가 있다.

* ### 목적과 데이터 특성에 맞는 모델은 무엇인가?
* ### 일반화 가능성은 어떠한가? (overfitting, underfitting의 가능성은?)
* ### 성능 측정의 지표는? 성능을 높이기 위해 어떻게 Feature Engineering 을 진행할 것인가?
* ### 제품 혹은 시스템에 모델을 적용할시 계산량이나 언어 특성에 관해 고려할 부분은 무엇인가?
* ### 모델/파라메터 업데이트 주기 및 방식은 어떻게 협의할 것인가?

## **Cross Validation**
### 구축된 모델의 일반화 가능성, 즉 overfitting, underfitting을 다루는 문제는 매우 중요하다. 이를 효과적으로 다룰 수 있는 방법이 Cross Vaildation이다. Cross Validation은 아래와 같은 순서로 진행된다. Testset은 최적화된 파라메터로 구축된 최종 모델의 성능을 파악하기 위해 단 1회만 사용한다. Dataset을 나눌 때 test_size 옵션으로 Train, Test의 비율을 설정할 수 있고, random_state로 seed 값을 지정할 수 있다.
* ### 전체 데이터셋을 3파트로 분류
    * ### 50%: Train set
    * ### 20%: Validation set (for grid search)
    * ### 30% Test set (to be used just once at the last moment)

![screenshot](.\img\35.png "35_Sh")

```
from sklearn.cross_validation import train_test_split

X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, random_state=23)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, random_state=11)
```
### 주로 이용되는 샘플링 방식은 아래와 같다. - Random Sampling - Stratified Sampling (Scikit-learn의 기본 옵션)

```
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)
```

## **Grid Search**
### 모델의 최적 파라메터를 도출하기 위해서 Kfold 와 Grid Search를 이용한다. Scikit-learn의 GridSearchCV 클래스를 이용하면 쉽게 진행할 수 있다. 해당 모델의 주요 파라메터는 아래와 같다.
* ### 참고문서: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
* ### 주요 파라메터 (C)
    * ### C 값 (기본값 = 1)
    * ### C 값이 작으면 Penalty 강해짐 (단순 모델)
    * ### C 값이 크면 Penalty 약해짐 (정규화 없어짐)
    * ### 보통 로그스케일로 지정(10배씩) = 0.01, 0.1, 1, 10
* ### penalty
    * ### L2: Ridge, 일반적으로 사용 (default)
    * ### L1: LASSO, 변수가 많아서 줄여야할 때 사용, 모델의 단순화 및 해석에 용이

![screenshot](.\img\36.png "36_Sh")

```
from sklearn.model_selection import GridSearchCV

# 파라메터 후보
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'penalty': ['l1', 'l2']}

# 그리드 서치 진행
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)        

# 최종 모델 성능 점검
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

grid_search.fit(X_train, y_train)
grid_search.score(X_test, y_test)
```

### 위와 동일한 코드는 아래와 같다. 원리 이해를 위해 코드 작성 및 실행을 해보는 것이 좋다.

```
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# SET default
best_score = 0

# iterataion
for r in ['l1', 'l2']:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        lm = LogisticRegression(penalty = r, C=C)
        scores = cross_val_score(lm, X_trainval, y_trainval, cv=5)
        score = np.mean(scores)
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'penalty': r}
```

[실습파일](https://github.com/songhunhwa/songhunhwa.github.com/tree/master/tutorial/tutorial_03 "모델 생성 및 학습")

# **04-5. 모델 평가**
----
## **평가 Matric**
### 회귀 및 분류 문제에 따라 평가하는 지표가 차이가 있다. 이번 프로젝트의 경우 이진 분류 문제에 해당하므로 Confusion Matrix 와 ROC Curve를 통해 모델을 평가하고 개선하는 작업을 진행하고자 한다. 참고로 평가지표를 설정할 경우에는 상황/환경, 우선순위 등에 따라 협의하에 선정된다.

* ### 회귀 문제

![screenshot](.\img\37.png "37_Sh")

* ### 분류 문제

![screenshot](.\img\38.png "38_Sh")

### Scikit-learn에서는 분류 문제와 관련한 성과 지표를 쉽게 확인할 수 있도록 아래 클래스를 제공한다.

```
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(grid_search.predict(X_test), y_test))
print(classification_report(grid_search.predict(X_test), y_test))
```

![screenshot](.\img\39.png "39_Sh")

```
from sklearn.metrics import roc_curve, auc

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, grid_search.predict(X_test))
roc_auc = auc(false_positive_rate, true_positive_rate)

fig = plt.figure(figsize=(6,4))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
```

[실습파일](https://github.com/songhunhwa/songhunhwa.github.com/tree/master/tutorial/tutorial_03 "모델 평가")

# **04-6. 모델 성과 개선**
----

### 첫 번째 모델의 성능을 baseline 으로 참고하여 전처리를 다시하거나 변수 선택 및 여러 모델 등을 적용해보며 퍼포먼스를 올리는 과정이 필요하다.

## **변수 스케일링 및 분포 변환**
### 모델의 성능을 높이기 위해 변수의 분포를 변환시키나 스케일을 조정하는 등 전처리 과정을 반복적으로 진행할 필요가 있다. 모델 특성 및 데이터 특성 등에 따라 적합한 전처리 과정이 다르므로 여러 번의 시도를 통해 모델의 성능을 올릴 필요가 있다.
* ### Standardization, Mean Normalizatoin, MinMax Scaling
* ### Log, Sqrt, Sqaure for distribution transformation

## **Upsampling & Downsampling**

### 타깃 변수의 그룹별 비율이 차이가 나는 경우, (예 결제자의 수가 매우 적고 비결제자의 수가 많은 경우) Imbalance 분포를 보이게 된다. 이와 같은 경우 모델이 학습을 위해 요구하는 충분한 데이터가 제공되지 않으므로 아래와 같은 방법을 고려해야 한다.

![screenshot](.\img\40.png "40_Sh")

* ### Collect More Data (if possible)
* ### Resampling the Dataset
    * ### **Oversampling** (no inofromation loss, perform better than undersampling, but overfitting issues)
    * ### **Undersampling** (help improve run time and storage problems, but information loss, biased dataset)
* ### Generate Synthetic Samples

## **Feature Selection**
### 모델에 적용할 input 변수를 잘 선정하는 것은 모델의 성능에 직접적인 영향을 미치는 과정이다. 아래의 방법을 이용해 모델의 성능을 높이는 데 유용한 변수를 선택할 필요가 있다.
* ### Univariate Selection (T-test, ANOVA, Coefficient and so on)
* ### Feature Importance (from Tree-based model)
* ### RFE (recursive feature elimination)
### 우선 Univariate Selection은 그룹내 분산이 작고 그룹간 분산이 클 경우 값이 커지는 F-value를 이용하여 변수를 선택한다. 각 변수마다 F값을 구해 F값이 큰 변수를 기준으로 변수를 선택하는 방법이다.
```
from sklearn.feature_selection import SelectKBest, f_classif

selectK = SelectKBest(score_func=f_classif, k=8)
X = selectK.fit_transform(X, y)
```

### ExtraTreesClassifier와 같은 트리 기반 모델은 Feature Importance 를 제공한다. 이 Feature Importance는 불확실도를 많이 낮출수록 증가하므로 이를 기준으로 변수를 선택할 수 있다.
```
from sklearn.ensemble import ExtraTreesClassifier

etc_model = ExtraTreesClassifier()
etc_model.fit(X, y)

print(etc_model.feature_importances_)
feature_list = pd.concat([pd.Series(X.columns), pd.Series(etc_model.feature_importances_)], axis=1)
feature_list.columns = ['features_name', 'importance']
feature_list.sort_values("importance", ascending =False)[:8]
```

### 마지막으로 RFE (recursive feature elimination)는 Backward 방식중 하나로, 모든 변수를 우선 다 포함시킨 후 반복해서 학습을 진행하면서 중요도가 낮은 변수를 하나씩 제거하는 방식이다.
```
from sklearn.feature_selection import RFE

model = LogisticRegression()
rfe = RFE(model, 8)
fit = rfe.fit(X, y)

print("Num Features: %d") % fit.n_features_
print("Selected Features: %s") % fit.support_
print("Feature Ranking: %s") % fit.ranking_
```

## **Model Selection**
### Scikit-learn은 전처리(스케일랑, feature selection, model selection)과 grid search 를 한번에 진행할 수 있도록 파이프라인 기능을 제공한다.
```
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline([('classifier', LogisticRegression())])

param_grid = [{'classifier': [SVC()], 
              'classifier__gamma': [0.01, 0.1, 1, 10, 100], 
              'classifier__C': [0.01, 0.1, 1, 10, 100]
              },

               {'classifier': [LogisticRegression()],
               'classifier__penalty': ['l1', 'l2'], 
               'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
               },

              {'classifier': [RandomForestClassifier()],
              'classifier__max_depth': [4, 6], # max_depth: The maximum depth of the tree.
              'classifier__n_estimators': [50, 100], # n_estimators: The number of trees in the forest.
              'classifier__min_samples_split': [50, 100]
              }] # min_samples_split: The minimum number of samples required to split an internal node       

grid = GridSearchCV(pipe, param_grid, scoring = 'roc_auc', cv=5)  
grid.fit(X_train, y_train)

print(grid.best_params_)
print(grid.best_score_)
```

[실습파일](https://github.com/songhunhwa/songhunhwa.github.com/tree/master/tutorial/tutorial_03 "모델 성과 개선")